## Pipeline Persistence

### Persistence
A pipeline persists its steps, context and dependencies. This example shows how those features of a pipeline work.

To set up:

In [ ]:
import numpy as np
import pandas as pd
from cortex import Cortex

Next, create a Cortex Builder instance:

In [ ]:
cortex = Cortex.local()
builder = cortex.builder()

Create the test pipeline:

In [ ]:
tp = builder.pipeline('test_pipeline')

Pipeline's have the method `to_camel()` that produces a textual representation of the state of the pipeline.

In [ ]:
tp.to_camel()

The [basic](./pipeline_basic.ipynb) function of a pipeline is to use a series of steps to transform a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe). To demonstrate persistence, we create a dataframe and add steps to our pipeline.

In [ ]:
# two columns of random numbers, indexed a through e
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
q = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])

# make a data frame by composing the columns together and labeling them
df = pd.DataFrame({'c1':s,'c2':q})

# display the result
df

Add a step to the pipeline:

In [ ]:
def add_one(pipeline, df):
    df['c1'] = df['c1'] + 1
    
tp.add_step(add_one)    

The camel representation of the test pipeline shows the Base 64 encoded `add_one` function.

In [ ]:
tp.to_camel()

## What is pipeline context?

A pipeline's context is a dictionary of values or functions that could be used to operate on the dataframe when each pipeline step is run.

In [ ]:
tp.set_context('one_median', df['c1'].astype(float).median())
tp.to_camel()

Now we add a step that uses the context:

In [ ]:
def standard_score(pipeline, df):
    mu = pipeline.get_context('one_median')
    df['c3'] = df['c1'] - mu # no need to divide, standard deviation is 1
    
tp.add_step(standard_score)

tp.run(df)

### Dependecies

Pipelines can be chained together, running one after another. The mechanism to do this is the pipeline dependencies function.

Create another pipeline:

In [ ]:
dtp = builder.pipeline('depends_on_test_pipeline')

def diff_c2_c3(pipeline, df):
    df['c4'] = df['c2'] - df['c3']
    

dtp.add_step(diff_c2_c3)

C3 must exist before this step is run, so `dtp` depends on `tp`:

In [ ]:
dtp.add_dependency(tp)

Now that a second pipeline is set up, run `dtp` to cause `tp` to run. 

In [ ]:
dtp.run(df)

## Builders vs Datasets

Datasets can have pipelines and datasets persist between kernel invocations, so pipelines associated with a dataset are persisted. Pipelines constructed from a builder are only persisted in memory. If the kernel restarts, the builders are deleted.

In [ ]:
data_set = builder.dataset('sample').title('Persistence Sample Data')\
    .from_df(df).build()

data_frame = data_set.as_pandas()
data_frame.tail()

In [ ]:
dsp = data_set.pipeline('dsp')
dsp.reset() # see below for an explanation of pipeline reset
dsp.add_step(add_one)
data_set.save()

When the Python kernel is reset, all in-memory data is cleared. However, the dataset, along with its pipeline, is persisted.

In [ ]:
# WARNING!! this cell resets the kernel, wiping out everything!
%reset -f

Notice that the test pipeline is gone:

In [ ]:
try:
    tp
except NameError:
    print('No tp')

Next we demonstrate that the `sample` dataset is persisted as is the `dsp` pipeline. In order to show how the pipeline persists, re-establish the required libraries and the Cortex builder:

In [ ]:
import numpy as np
import pandas as pd
from cortex import Cortex

cortex = Cortex.local()
builder = cortex.builder()

Get the dataset, using the name we previousily specified:

In [ ]:
data_set = cortex.dataset('sample')

Get the pipeline, again using the name we previousily specified:

In [ ]:
new_pipeline = data_set.pipeline('dsp')

Get the dataframe from the dataset and then run the pipeline. 

In [ ]:
df = data_set.as_pandas()
new_pipeline.run(df)

Notice that the `add_one` function is still present in the `new_pipeline`.

## Reset() affects pipeline state

Without parameters, reset removes all steps in the pipline. 

In [ ]:
new_pipeline.to_camel()

In [ ]:
new_pipeline.reset()

new_pipeline.to_camel()

You can also delete all context and dependencies. 

In [ ]:
another_new_pl = data_set.pipeline('dsp')

Add a context:

In [ ]:
another_new_pl.set_context('for_example','a string')

Add a dependency:

In [ ]:
tp = builder.pipeline('test_pipeline')
another_new_pl.add_dependency(tp)

another_new_pl.to_camel()

Next reset everything, including pipeline steps, dependencies, and context:

In [ ]:
another_new_pl.reset(reset_deps=True, reset_context=True)

another_new_pl.to_camel()